<a href="https://colab.research.google.com/github/andhika7/deep_multiple_linier_regression/blob/main/my_model_prediksi_performance_komputer_multiple_linier_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [ ]:
names = ['VENDOR','MODEL_NAME','MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX', 'PRP', 'ERP' ];
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data',names=names)
data.head(5)

###Attribute Information:

1. vendor name: 30 (adviser, amdahl,apollo, basf, bti, burroughs, c.r.d, cambex, cdc, dec, dg, formation, four-phase, gould, honeywell, hp, ibm, ipl, magnuson, microdata, nas, ncr, nixdorf, perkin-elmer, prime, siemens, sperry, sratus, wang)
2. Model Name: many unique symbols
3. MYCT: machine cycle time in nanoseconds (integer)
4. MMIN: minimum main memory in kilobytes (integer)
5. MMAX: maximum main memory in kilobytes (integer)
6. CACH: cache memory in kilobytes (integer)
7. CHMIN: minimum channels in units (integer)
8. CHMAX: maximum channels in units (integer)
9. PRP: published relative performance (integer)
10. ERP: estimated relative performance from the original article (integer)

### preprocessing

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
plt.figure(figsize=(8,5))
sns.heatmap(data.corr(), annot=True) #seaborn
plt.title('Correlation between the columns')
plt.show()

### split dataset ke dependent dan independent features

In [ ]:
# attribute 'model' tidak dipakai karena tidak mempengaruhi prediksi
X = data.iloc[:,[0,2,3,4,5,6,7,8]].values
y = data.iloc[:, 9].values

In [ ]:
X.shape

### mengubah kolom vendor menjadi tensor agar dapat terbaca program

In [ ]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
enc=LabelEncoder()
X[:,0]=enc.fit_transform(X[:,0])
# onenc=OneHotEncoder(categorical_features=[0])
onenc=ColumnTransformer([("VENDOR",OneHotEncoder(),[0])], remainder="passthrough")
X=onenc.fit_transform(X).toarray()

In [ ]:
X

In [ ]:
X=X[:,1:]

In [ ]:
X.shape

### split dataset menjadi data training dan data test

In [ ]:
#split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

### standarisasi data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()

X_train=sc_x.fit_transform(X_train)
X_test=sc_x.transform(X_test)

Membangun deep model dengan 3 buah hidden layer dengan masing2 30 neuron

In [ ]:
# model
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential() # model nama variabel
model.add(Dense(X.shape[1], activation='relu')) # input layer (array ke 1)
model.add(Dense(30, activation='relu')) # hidden layer 1
model.add(Dense(30, activation='relu')) # hidden layer 2
model.add(Dense(30, activation='relu')) # hidden layer 3
model.add(Dense(1)) # output 1 neuron

model.compile(optimizer='adam', loss='mae') # gabung menjadi 1, dengan optimizer adam yg umum digunakan

In [ ]:
history = model.fit(X_train, y_train,  # variabel history digunakan menyimpan hasil pembelajaran
                    validation_split=0.2, # validasi 20% dari data training
                    epochs=100) # belajar sebanyak 100 kali

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('mean absolute error')
plt.show()

In [ ]:
Y_pred = model.predict(X_test)

### mengukur nilai prediksi dengan 3 buah parameter: MAE, MSE, dan R2 score

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import metrics

print("MAE: ", (metrics.mean_absolute_error(Y_pred, y_test)))
print("MSE: ", (metrics.mean_squared_error(Y_pred, y_test)))
print("R2 score: ", (metrics.r2_score(Y_pred, y_test)))

In [ ]:
sns.regplot(x=Y_pred, y=y_test)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Actual vs predicted')
plt.show()